# Cleaning Melbourne city data - other features
This script cleans the data on a number of different feature types for which there is data for Melbourne.
All data is from: https://data.melbourne.vic.gov.au/  

#### Yearly Building Data
This dataset has info from the City of Melbourne's Census of Land Use and Employment (CLUE). It contains annual counts of the number of buildings, their uses, number of floors, accessibility, and location coordinates. 


#### Bike Dock Locations & Capacities
Until the end of 2019 Melbourne had a bike share system in the city, and this data set includes bike dock locations and capacities.  
Also additional dataset: Melbourne Bike Share Station Readings 2011-2017, which might be useful

#### Landmarks and Points of Interest
This dataset provides the locations of various landmarks and points of interest in the city. 

#### Street lighting
Includes all city owned lighting with the wattage, light type, and location.
Downloaded from: https://data.melbourne.vic.gov.au/City-Council/Feature-Lighting-including-light-type-wattage-and-/4j42-79hg 
Maybe street lighting would be better: https://data.melbourne.vic.gov.au/City-Council/Street-lights-with-emitted-lux-level-council-owned/hg8j-vcww

####  Street furniture

Ideas on other data available at open data website:
* social indicators (affluence of area?); tree canopies (green space?; live music venues; Super Sunday bike count; 
* Metro Train Stations with Accessibility Information (landmarks only has Railway Station', 'Transport Terminal', 'Marina', 'Bridge' under transport sub-themes - mero stations might feature in transport terminal but would need to check); playgrouns? Cafe, restaurant, bistro seats; Bar, tavern, pub patron capacity; taxi ranks; public toilets; bus stop locations; 

#### Code initialisation

In [2]:
import pandas as pd
import numpy as np
# import seaborn as sns
import os
import folium 
from datetime import date, datetime

# Memorial and sculptures

In [6]:
# Read in data
memorials = pd.read_csv('../Data/public-memorials-and-sculptures.csv')
split_data = memorials['Co-ordinates'].str.strip(')').str.strip('(').str.split(', ')
memorials['Latitude'] = split_data.apply(lambda x: x[0])
memorials['Longitude'] = split_data.apply(lambda x: x[1])
memorials.drop('Co-ordinates', axis = 1, inplace = True)
memorials.to_csv('../Cleaned_data/memorials_clean.csv', header = memorials.columns, index=False)

# Buildings

In [ ]:
# Read in data
buildings = pd.read_csv('../Data/Buildings_with_name__age__size__accessibility__and_bicycle_facilities.csv')

# Remove columns not needed
buildings.drop(['Block ID', 'Property ID', 'Base property ID', 'Building name', 'Construction year', 'Location',
                'Street address', 'CLUE small area', 'Refurbished year', 'Has showers', 'Accessibility type description'], axis = 1, inplace = True)
# Rename columns
buildings.rename({'Census year': 'year', 'Number of floors (above ground)': 'n_floors', 
                 'Predominant space use': 'building_use', 'Accessibility type': 'access_type',
                 'Accessibility rating': 'access_rating', 'Bicycle spaces': 'bike_spaces',
                 'x coordinate': 'Longitude', 'y coordinate': 'Latitude'}, axis =1, inplace = True)

# Drop 130 buildings that don't have location coordinates
buildings.dropna(subset = ['Longitude'], axis = 0, inplace = True)

# Keep only buildings from 2010 onwards (as this is when camera data is for)
buildings = buildings[buildings['year'] >= 2010]

# Shorten variable names for access type, and convert the variable to categorical
buildings.access_type = buildings.access_type.replace({'Not determined or not applicable': np.nan, 'Low level of accessibility': 'low', 
                               'Moderate level of accessibility': 'moderate', 'High level of accessibility': 'high'})
cat_type = pd.CategoricalDtype(categories = ['low', 'moderate', 'high'], ordered = True)
buildings.access_type = buildings.access_type.astype(cat_type)

#### Check for variables with missing values
Columns which have a high proportion of NULL should perhaps be dropped

In [ ]:
# round(buildings.isnull().sum()/len(buildings) * 100,1)
# buildings['access_type'].value_counts(dropna = False)

In [ ]:
#see correlations
# sns.heatmap(buildings.corr(), annot = True);

#### Save to file

In [ ]:
buildings.to_csv('../Cleaned_data/buildings_clean.csv', header = buildings.columns, index=False)

# Landmarks & Places of Interest
"This dataset contains a description and co-ordinates of places of interest within the City of Melbourne.

Themes include: Community Use, Education Centre, Health Services, Leisure/Recreation, Mixed Use, Office, Place Of Assembly, Place of Worship, Purpose Built, Retail, Transport, Vacant Land

Sub-themes include: Art Gallery/Museum, Church, Function/Conference/Exhibition Centre, Informal Outdoor Facility (Park/Garden/Reserve), Major Sports & Recreation Facility, Office, Public Buildings, Public Hospital, Railway Station, Retail/Office/Carpark, Tertiary (University), Theatre Live"

In [6]:
# Read in data
landmarks = pd.read_csv('../Data/Landmarks_and_places_of_interest__including_schools__theatres__health_services__sports_facilities__places_of_worship__galleries_and_museums..csv')

# Check whether any of the variables have missing data
landmarks.isnull().sum()

# Split co-ordinates into latitude and longitude columns like building data is in
split_data = landmarks['Co-ordinates'].str.strip(')').str.strip('(').str.split(', ')
landmarks['Latitude'] = split_data.apply(lambda x: x[0])
landmarks['Longitude'] = split_data.apply(lambda x: x[1])
landmarks.drop('Co-ordinates', axis = 1, inplace = True)

# Rename columns
landmarks.rename({'Theme': 'theme', 'Sub Theme': 'sub_theme', 'Feature Name': 'feature_name'}, axis =1, inplace = True)

#### Print a summary of the kind of landmarks present in the dataset by theme and subtheme

In [644]:
# landmarks.groupby(['theme', 'sub_theme'])['sub_theme'].count()

#### Save to file

In [26]:
landmarks.to_csv('../Cleaned_data/landmarks_clean.csv', header = landmarks.columns, index=False)

# Bike Share Locations and capacity
### Even though this program ended in 2019, the docks were in use for the majority of years that this analysis will use
"This dataset show the historical Melbourne Bike Share docks. This program came to an end in November 2019.

Contains the bike share dock locations that were deployed across Melbourne as part of the Melbourne Bike Share Program.

Melbourne Bike Share is a joint RACV/Victorian Government bicycle hire scheme. It allows commuters to hire a bike from a dock location and return it to another dock location in the city. This dataset contains the bike share dock locations and capacity across the city."

In [14]:
bikes = pd.read_csv('../Data/Bike_Share_Dock_Locations.csv')

#### Clean bike share data

In [15]:
# Check for variables with missing values
bikes.isnull().sum()
# Remove unneeded columns
bikes.drop(['rental_method', 'location', 'name'], axis =1, inplace= True)
# Rename columns
bikes.rename({'lat': 'Latitude', 'lon': 'Longitude'}, axis =1, inplace = True)

In [27]:
# # Plot
# melbourne_map = folium.Map(location=[bikes.Latitude.mean(),
#                            bikes.Longitude.mean()], zoom_start=13, control_scale=True)
# for index, location_info in bikes.iterrows():
#     folium.Marker([location_info["Latitude"], location_info["Longitude"]]).add_to(melbourne_map)
# melbourne_map

#### Save to file

In [20]:
bikes.to_csv('../Cleaned_data/bikes_clean.csv', header = bikes.columns, index=False)

# City Lighting
"This dataset contains information such as location, lighting type and wattage of feature lighting across City of Melbourne.

Feature lights are usually found around high profile areas of the city."

In [21]:
lights = pd.read_csv('../Data/Feature_Lighting__including_light_type__wattage_and_location_.csv')

#### Clean lighting dataset

In [22]:
# Check for variables with missing values
lights.isnull().sum()
# Remove unneeded columns
lights.drop(['asset_number', 'asset_description', 'mounting_type_lupvalue', 'location'], axis = 1, inplace = True)
# Rename columns
lights.rename({'lat': 'Latitude', 'lon': 'Longitude'}, axis =1, inplace = True)
lights

,lamp_type_lupvalue,lamp_rating_w,Latitude,Longitude
0,13.0,70.0,-37.825148,144.947298
1,9.0,NaN,-37.822606,144.946367
2,13.0,35.0,-37.818873,144.943174
3,5.0,35.0,-37.824647,144.945689
4,9.0,10.0,-37.824407,144.947251
...,...,...,...,...
8559,NaN,NaN,-37.820395,144.945234
8560,9.0,NaN,-37.818989,144.972787
8561,13.0,35.0,-37.819837,144.959517
8562,NaN,NaN,-37.823714,144.951896


In [23]:
# Plot
melbourne_map = folium.Map(location=[lights.Latitude.mean(),
                           lights.Longitude.mean()], zoom_start=13, control_scale=True)
for index, location_info in lights.iterrows():
    folium.Marker([location_info["Latitude"], location_info["Longitude"]]).add_to(melbourne_map)
melbourne_map

#### Save to csv

In [5]:
lights.to_csv('../Cleaned_data/lights_clean.csv', header = lights.columns, index=False)

## Street Infrastructure
"The City of Melbourne owns and maintains various objects and pieces of equipment installed on streets and roads for various purposes. This dataset includes Barbeques, Bicycle Rails, Bin Corrals, Bollards, Drinking Fountains, Floral Crate/Planter Boxs, Hoops, Horse Troughs, Information Pillars, Litter Bins, Picnic Setting, Seats."

In [10]:
street_inf = pd.read_csv('../Data/Street_furniture_including_bollards__bicycle_rails__bins__drinking_fountains__horse_troughs__planter_boxes__seats__barbecues.csv')

#### Clean street infrastructure dataset

In [11]:
# Check for variables with missing values
street_inf.isnull().sum()

# Drop unneeded columns
street_inf.drop(['GIS_ID', 'DESCRIPTION', 'MODEL_NO', 'MODEL_DESCR', 'DIVISION', 'COMPANY',
                'LOCATION_DESC', 'EVALUATION_DATE', 'EASTING', 'NORTHING', 'UploadDate'], axis = 1, inplace = True)

# Split coordinates into lat/long coordinate columns
split_data = street_inf['CoordinateLocation'].str.strip(')').str.strip('(').str.split(', ')
street_inf['Latitude'] = split_data.apply(lambda x: x[0])
street_inf['Longitude'] = split_data.apply(lambda x: x[1])
street_inf.drop('CoordinateLocation', axis = 1, inplace = True)

# Rename columns to be only lowercase
street_inf.rename({'ASSET_CLASS': 'asset_class', 'ASSET_TYPE': 'feature', 'CONDITION_RATING': 'condition_rating'},
                 axis = 1, inplace = True)

# Drop the asset class as the only value present is street furniture
street_inf.drop('asset_class', axis =1, inplace = True)

In [30]:
street_inf.to_csv('../Cleaned_data/street_inf_clean.csv', header = street_inf.columns, index=False)